## 2. Tensorflow Low Level API 

### 2.4. 图和会话Graphs and Sessions

**数据流图的优点Why dataflow graphs**

---

![](https://www.tensorflow.org/images/tensors_flowing.gif)


数据流[Dataflow](https://en.wikipedia.org/wiki/Dataflow_programming)是一种用于并行计算的常用编程模型。

数据流为Tensorflow程序提供多项优势：

1. **并行处理Parallelism**：通过使用显式的边(edges)来表示操作(op)之间的依赖关系，系统可以轻松识别能够并行执行的操作(op)。
2. **分布式执行Distributed execution**：通过使用显式的边(edges)来表示操作(op)之间流动的值，TensorFlow可以将您的程序划分到不同机器上的多台设备（CPU、GPU 和 TPU）。TensorFlow将在这些设备之间进行必要的通信和协调。
3. **编译Compilation**：TensorFlow 的[XLA 编译器](https://www.tensorflow.org/performance/xla/index)可以使用数据流图中的信息生成更快的代码，例如将相邻的操作融合到一起。
4. **可移植性Portability**：语言独立性，可以使用 Python 构建数据流图，将其存储在[SavedModel](https://www.tensorflow.org/guide/saved_model)中，并使用`C++`程序进行恢复，从而实现低延迟的推理。


**数据流图类定义What is [tf.Graph](https://www.tensorflow.org/api_docs/python/tf/Graph)**

---

**计算流图**是一系列节点（操作[tf.Operation](https://www.tensorflow.org/api_docs/python/tf/Operation)）和边（张量[tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor)）的有向图，由两种类型对象组成：

+ [tf.Operation](https://www.tensorflow.org/api_docs/python/tf/Operation)(简称“op”)：图的节点。操作表示消耗和生成张量的数学计算；或者表示数据输入(feed in)的起点/输出(push out)的终点，或者表示读取/写入持久变量(persistent variable)的终点
+ [tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor)：图的边。操作(op)之间的输入/输出关系，即“张量”(tensor)。

**计算流图类[tf.Graph](https://www.tensorflow.org/api_docs/python/tf/Graph)**包含两种信息：（记）

+ **图结构Graph Structure**：图的节点和边，表示了各个操作组合在一起的方式，但不规定它们的使用方式。图结构与汇编代码类似：检查图结构可以传达一些有用的信息，但它不包含源代码传达的所有实用上下文信息。
+ **图集合Graph Collections**： TensorFlow提供了一种在[tf.Graph](https://www.tensorflow.org/api_docs/python/tf/Graph)中存储元数据集合的通用机制。
    + [tf.add_to_collection](https://www.tensorflow.org/api_docs/python/tf/add_to_collection)函数：关联对象列表与一个键（其中[tf.GraphKeys](https://www.tensorflow.org/api_docs/python/tf/GraphKeys)定义了部分标准键）;
    + [tf.get_collection](https://www.tensorflow.org/api_docs/python/tf/get_collection)函数：获取某个键关联的所有对象。
 
TensorFlow库的许多部分会使用图集合：例如，当您创建[tf.Variable](https://www.tensorflow.org/api_docs/python/tf/Variable)时，系统会默认将其添加到表示“全局变量”和“可训练变量”的集合中。当您后续创建 [tf.train.Saver](https://www.tensorflow.org/api_docs/python/tf/train/Saver)或[tf.train.Optimizer](https://www.tensorflow.org/api_docs/python/tf/train/Optimizer)时，这些集合中的变量将用作默认参数。

```
 |  A `Graph` contains a set of
 |  `tf.Operation` objects,
 |  which represent units of computation; and
 |  `tf.Tensor` objects, which represent
 |  the units of data that flow between operations.
```


**数据流图类实例构建Building a [tf.Graph](https://www.tensorflow.org/api_docs/python/tf/Graph)**

---

TensorFlow提供了一个**默认图Default Graph**，此图是同一上下文中的所有Tensorflow API函数的隐式参数。所有构建操作对象[tf.Operation](https://www.tensorflow.org/api_docs/python/tf/Operation)和张量对象[tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor)的Tensorflow API函数都将构造对象并且将其添加至默认图;

```
 |  A default `Graph` is always registered, and accessible by calling
 |  `tf.get_default_graph`.
 |  To add an operation to the default graph, simply call one of the functions
 |  that defines a new `Operation`:
 |
 |  ``python
 |  c = tf.constant(4.0)
 |  assert c.graph is tf.get_default_graph()
 |  ``
 |
 |  Another typical usage involves the
 |  `tf.Graph.as_default`
 |  context manager, which overrides the current default graph for the
 |  lifetime of the context:
 |
 |  ``python
 |  g = tf.Graph()
 |  with g.as_default():
 |    # Define operations and tensors in `g`.
 |    c = tf.constant(30.0)
 |    assert c.graph is g
 |  ``
```

In [1]:
import tensorflow as tf

print(tf.get_default_graph())

c = tf.constant(4.)
assert c.graph is tf.get_default_graph()

g = tf.Graph()
with g.as_default():
    # Define operations and tensors in `g`.
    c = tf.constant(30.)
    assert c.graph is g
    
with tf.Graph().as_default() as g:
    c = tf.constant(5.)
    assert c.graph is g

例子：

+ Calling `tf.constant(42.0)` creates a single [tf.Operation]() that produces the value 42.0, adds it to the default graph, and returns a [tf.Tensor]() that represents the value of the constant.

+ Calling `tf.matmul(x, y)` creates a single [tf.Operation]() that multiplies the values of [tf.Tensor]() objects `x` and `y`, adds it to the default graph, and returns a [tf.Tensor]() that represents the result of the multiplication.

+ Executing `v = tf.Variable(0)` adds to the graph a [tf.Operation]() that will store a writeable tensor value that persists between [tf.Session.run](https://www.tensorflow.org/api_docs/python/tf/Session#run) calls. The [tf.Variable]() object wraps this operation, and can be used [like a tensor]()(Tensor-like object), which will read the current value of the stored value. The [tf.Variable]() object also has methods such as [tf.Variable.assign](https://www.tensorflow.org/api_docs/python/tf/Variable#assign) and [tf.Variable.assign_add](https://www.tensorflow.org/api_docs/python/tf/Variable#assign_add) that create [tf.Operation]() objects that, when executed, update the stored value. (See [Variables]() for more information about variables.)
 + Calling [tf.train.Optimizer.minimize](https://www.tensorflow.org/api_docs/python/tf/train/Optimizer#minimize) will add operations and tensors to the default graph that calculates gradients, and return a [tf.Operation]() that, when run, will apply those gradients to a set of variables.
 
[tf.Tensor]: https://www.tensorflow.org/api_docs/python/tf/Tensor
[tf.Operation]: https://www.tensorflow.org/api_docs/python/tf/Operation
[tf.Variable]: https://www.tensorflow.org/api_docs/python/tf/Variable
 
大多数程序仅依赖于默认图。尽管如此，请参阅[处理多个图Dealing with multiple graphs](https://www.tensorflow.org/guide/graphs?hl=en#programming_with_multiple_graphs)了解更加高级的用例。高阶 API（比如[tf.estimator.Estimator](https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator) API）可替您管理默认图，并且还具有其他功能，例如创建不同的图以用于训练和评估。

>注意：调用 TensorFlow API 中的大多数函数只会将操作和张量添加到默认图中，而**不会**执行实际计算。您应组织这些api函数，直到拥有表示整个计算的 [tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor)或[tf.Operation](https://www.tensorflow.org/api_docs/python/tf/Operation)（例如执行梯度下降法的一步），然后将该对象传递给[tf.Session](https://www.tensorflow.org/api_docs/python/tf/Session)以执行计算。更多详情请参阅“在[tf.Session](https://www.tensorflow.org/api_docs/python/tf/Session)中执行图”部分。


**操作的名称Naming operations**（记）

---

[tf.Graph](https://www.tensorflow.org/api_docs/python/tf/Graph)对象会为其包含的[tf.Operation](https://www.tensorflow.org/api_docs/python/tf/Operation)对象定义一个**命名空间**。

操作名称格式：`{op_name}`+`(_{number})`;

张量名称格式：`{op_name}`+`(_{number})`+`:`+`{output_index}`;

重命名操作的方法：（记）

1. 使用所有API函数的`name`参数；For example, `tf.constant(42.0, name="answer")` creates a new [tf.Operation](https://www.tensorflow.org/api_docs/python/tf/Operation) named "answer" and returns a [tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor) named "answer:0"
2. [tf.name_scope](https://www.tensorflow.org/api_docs/python/tf/name_scope)函数：为在特定上下文中创建的所有操作添加名称作用域前缀；当前名称作用域前缀是一个用 "/" 分隔的，包含所有活跃[tf.name_scope](https://www.tensorflow.org/api_docs/python/tf/name_scope)上下文管理器（会叠加、嵌套）的名称列表。

In [2]:
c_0 = tf.constant(0, name="c")  # operation named "c"

# Already-used names will be "uniquified".
c_1 = tf.constant(2, name="c")  # operation named "c_1"

# Name scopes add a prefix to all operations created in the same context.
with tf.name_scope("outer"):
    c_2 = tf.constant(2, name="c")  # operation named "outer/c"
    
    # Name scopes nest like paths in a hierarchical file system.
    with tf.name_scope("inner"):
        c_3 = tf.constant(3, name="c")   # operation named "outer/inner/c"
        
    # Exiting a name scope context will return to the previous prefix.
    c_4 = tf.constant(4, name="c")  # operation named "outer/c_1"
    
    # Already-used name scope will be "uniquified"
    with tf.name_scope("inner"):
        c_5 = tf.constant(5, name="c")  # operation named "outer/inner_1/c"


**部署操作(ops)至不同设备Placing operations on different devices**

---

（分布式模型的内容，稍后添加）

**Tensor-like对象Tensor-like Objects**（记）

---

Tensorflow API函数以Tensor-like object对输入，而不是张量对象[tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor)，并隐式调用[tf.convert_to_tensor](https://www.tensorflow.org/api_docs/python/tf/convert_to_tensor)方法将其转换成张量[tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor)；

Tensor-like object包含：

1. [tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor);
2. [tf.Variable](https://www.tensorflow.org/api_docs/python/tf/Variable);
3. [numpy.ndarray](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html);
4. `list`(and list of tensor-like objects)
5. Scalar Python types: `bool`, `int`, `float`, `str`, `complex`;

可以使用[tf.register_tensor_conversion_function](https://www.tensorflow.org/api_docs/python/tf/register_tensor_conversion_function)注册其他Tensor-like类型。

> 注意：默认情况下，每次您使用同一个Tensor-like对象时，TensorFlow将创建新的[tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor)。如果Tensor-like对象很大（例如包含一组训练样本的`numpy.ndarray`），且您多次使用该对象，则可能会耗尽内存。要避免出现此问题，请在类张量对象上手动调用[tf.convert_to_tensor](https://www.tensorflow.org/api_docs/python/tf/convert_to_tensor)一次，并使用返回的[tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor)。



**在[tf.Session](https://www.tensorflow.org/api_docs/python/tf/Session)中执行计算流图Executing a graph in a [tf.Session](https://www.tensorflow.org/api_docs/python/tf/Session)**

---

**创建[tf.Session]()**

低阶TensorFlow API可以为当前默认图创建一个[tf.Session]()，如下所示

```
# Create a default in-process session.
with tf.Session() as sess:
    # ...
 
# Create a remote session.
with tf.Session("grpc://example.org:2222"):
    # ...

```

由于[f.Session]()物理资源（例如 GPU 和网络连接），因此通常（在`with`代码块中）用作上下文管理器，并在您退出代码块时自动关闭会话。您也可以在不使用`with`代码块的情况下创建会话，但应在完成会话时明确调用[tf.Session.close](https://www.tensorflow.org/api_docs/python/tf/Session#close)以便释放资源。

> 注意：较高阶的 API（例如[tf.train.MonitoredTrainingSession](https://www.tensorflow.org/api_docs/python/tf/train/MonitoredTrainingSession)或[tf.estimator.Estimator](https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator)）将为您创建和管理[tf.Session](https://www.tensorflow.org/api_docs/python/tf/Session)。这些 API 接受可选的**target**和**config**参数（直接接受，或作为[tf.estimator.RunConfig](https://www.tensorflow.org/api_docs/python/tf/estimator/RunConfig)对象的一部分），并具有相同的含义，如下所示。

[tf.Session.init](https://www.tensorflow.org/api_docs/python/tf/Session#__init__)接受三个可选参数：

+ **target**。 如果将此参数留空（默认设置），会话将仅使用本地机器中的设备。但是，您也可以指定`grpc://`网址，以便指定 TensorFlow 服务器的地址，这使得会话可以访问该服务器控制的机器上的所有设备。请参阅[tf.train.Server](https://www.tensorflow.org/api_docs/python/tf/train/Server)以详细了解如何创建 TensorFlow 服务器。例如，在常见的**图间复制between-graph replication**配置中，[tf.Session]()连接到[tf.train.Server](https://www.tensorflow.org/api_docs/python/tf/train/Server)的流程与客户端相同。分布式 TensorFlow 部署指南介绍了其他常见情形。

+ **graph**。 默认情况下，新的[tf.Session](https://www.tensorflow.org/api_docs/python/tf/Session)将绑定到当前的默认图，并且仅能够在当前的默认图中运行操作。如果您在程序中使用了多个图（更多详情请参阅[使用多个图进行编程Programming with multiple graphs](https://www.tensorflow.org/guide/graphs?hl=en#programming_with_multiple_graphs)），则可以在构建会话时指定明确的[tf.Graph](https://www.tensorflow.org/api_docs/python/tf/Graph)。

+ **config**。 此参数允许您指定一个控制会话行为的[tf.ConfigProto](https://www.tensorflow.org/api_docs/python/tf/ConfigProto)。例如，部分配置选项包括：
    + `allow_soft_placement`。将此参数设置为`True`可启用“软”设备放置算法，该算法会忽略尝试将仅限 CPU 的操作分配到 GPU 设备上的 [tf.device]()语句，并将这些操作放置到 CPU 上。
    + `cluster_def`。使用分布式 TensorFlow 时，此选项允许您指定要在计算中使用的机器，并提供作业名称、任务索引和网络地址之间的映射。详情请参阅[tf.train.ClusterSpec.as_cluster_def](https://www.tensorflow.org/api_docs/python/tf/train/ClusterSpec#as_cluster_def)。
    + `graph_options.optimizer_options`。在执行图之前使您能够控制 TensorFlow 对图实施的优化。
    + `gpu_options.allow_growth`。将此参数设置为`True`可更改 GPU 内存分配器，使该分配器逐渐增加分配的内存量，而不是在启动时分配掉大多数内存。

    
**使用[tf.Session.run](https://www.tensorflow.org/api_docs/python/tf/Session#run)执行操作**

**[tf.Session.run](https://www.tensorflow.org/api_docs/python/tf/Session#run)要求您指定一组**fetch**，这些fetch可确定返回值，并且可能是[tf.Operation](https://www.tensorflow.org/api_docs/python/tf/Operation)、[tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor)或tensor-like类型，**例如[tf.Variable](https://www.tensorflow.org/api_docs/python/tf/Variable)。这些fetch决定了必须执行哪些子图（属于整体[tf.Graph](https://www.tensorflow.org/api_docs/python/tf/Graph)）以生成结果：该子图包含 fetch 列表中指定的所有操作，以及其输出用于计算 fetch 值的所有操作。例如，以下代码段说明了[tf.Session.run](https://www.tensorflow.org/api_docs/python/tf/Session#run)的不同参数如何导致执行不同的子图：
    
[tf.Session]: https://www.tensorflow.org/api_docs/python/tf/Session

In [4]:
x = tf.constant([[37. , -23.], [1. , 4.]])
w = tf.Variable(tf.random_uniform([2, 2]))
y = tf.matmul(x, w)
output = tf.nn.softmax(y)
init_op = w.initializer

with tf.Session() as sess:
    # Run the initializer on `w`.
    sess.run(init_op)
    
    # Evaluate `output`. `sess.run(output)` will return a NumPy array containing
    # the result of the computation.
    print(sess.run(output))
    
    # Evaluate `y` and `output`. Note that `y` will only be computed once, and its
    # result used both to return `y_val` and as an input to the `tf.nn.softmax()`
    # op. Both `y_val` and `output_val` will be NumPy arrays.
    y_val, output_val = sess.run([y, output])
    

[[0.39389583 0.6061042 ]
 [0.52594227 0.47405776]]


[tf.Session.run](https://www.tensorflow.org/api_docs/python/tf/Session#run)也可以选择接受**feed**字典，该字典是从[tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor)对象（通常是[tf.placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder)张量）到在执行时会替换这些张量的值（通常是 Python 标量、列表或 NumPy 数组）的映射。例如：

In [7]:
# Define a placeholder that expects a vector of three floating-point values, and a computation that depends on it.
x = tf.placeholder(tf.float32, shape=[3])
y = tf.square(x)

with tf.Session() as sess:
    # Feeding a value changes the result that is returned when you evaluate `y`.
    print(sess.run(y, {x: [1., 2., 3.]}))  # [1., 4., 9.]
    print(sess.run(y, {x: [0., 0., 5.]}))  # [0., 0., 25.]
    
    # Raises <a href="../api_docs/python/tf/errors/InvalidArgumentError"><code>tf.errors.InvalidArgumentError</code></a>, because you must feed a value for
    # a `tf.placeholder()` when evaluating a tensor that depends on it.
    # sess.run(y)
    
    # Raises `ValueError`, because the shape of `37.0` does not match the shape
    # of placeholder `x`.
    # sess.run(y, {x: 37.0})

[1. 4. 9.]
[ 0.  0. 25.]


[tf.Session.run](https://www.tensorflow.org/api_docs/python/tf/Session#run) also accepts an optional options argument that enables you to specify options about the call, and an optional run_metadata argument that enables you to collect metadata about the execution. For example, you can use these options together to collect tracing information about the execution:

In [9]:
y = tf.matmul([[37., -23.], [1., 4.]], tf.random_uniform([2, 2]))

with tf.Session() as sess:
    # Define options for the `sess.run()` call.
    options = tf.RunOptions()
    options.output_partition_graphs = True
    options.trace_level = tf.RunOptions.FULL_TRACE
    
    # Define a container for the returned metadata.
    metadata = tf.RunMetadata()
    
    sess.run(y, options=options, run_metadata=metadata)
    
    # Print the subgraphs that executed on each device.
    print(metadata.partition_graphs)
    
    # Print the timings of each operation that executed.
    print(metadata.step_stats)

[node {
  name: "MatMul_3/a"
  op: "Const"
  device: "/job:localhost/replica:0/task:0/device:CPU:0"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 2
          }
          dim {
            size: 2
          }
        }
        tensor_content: "\000\000\024B\000\000\270\301\000\000\200?\000\000\200@"
      }
    }
  }
}
node {
  name: "random_uniform_3/shape"
  op: "Const"
  device: "/job:localhost/replica:0/task:0/device:CPU:0"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
          dim {
            size: 2
          }
        }
        tensor_content: "\002\000\000\000\002\000\000\000"
      }
    }
  }
}
node {
  name: "random_uniform_3/RandomUniform"
  op: "RandomUniform"
  input: "random_uniform_3/shape"
  

**可视化计算流图Visualizing your graph**

---

TensorFlow includes tools that can help you to understand the code in a graph. The graph visualizer is a component of TensorBoard that renders the structure of your graph visually in a browser. The easiest way to create a visualization is to pass a [tf.Graph](https://www.tensorflow.org/api_docs/python/tf/Graph) when creating the [tf.summary.FileWriter](https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter):

```
# Build your graph.
x = tf.constant([[37.0, -23.0], [1.0, 4.0]])
w = tf.Variable(tf.random_uniform([2, 2]))
y = tf.matmul(x, w)
# ...
loss = ...
train_op = tf.train.AdagradOptimizer(0.01).minimize(loss)

with tf.Session() as sess:
  # `sess.graph` provides access to the graph used in a <a href="./../api_docs/python/tf/Session"><code>tf.Session</code></a>.
  writer = tf.summary.FileWriter("/tmp/log/...", sess.graph)

  # Perform your computation...
  for i in range(1000):
    sess.run(train_op)
    # ...

  writer.close()
```

(未完待续)


**使用多个图编程Programming with multiple graphs**

---

> 注意：训练模型时，整理代码的一种常用方法是使用一个图训练模型，然后使用另一个图对训练过的模型进行评估或推理。在许多情况下，推理图与训练图不同：例如，丢弃和批次标准化等技术在每种情形下使用不同的操作。此外，默认情况下，tf.train.Saver 等实用程序使用 tf.Variable 对象的名称（此类对象的名称基于底层 tf.Operation）来识别已保存检查点中的每个变量。采用这种方式编程时，您可以使用完全独立的 Python 进程来构建和执行图，或者在同一进程中使用多个图。此部分介绍了如何在同一进程中使用多个图。

如上所述，TensorFlow 提供了一个“默认图”，此图明确传递给同一上下文中的所有 API 函数。对于许多应用而言，单个图便已足够。但是，TensorFlow 还提供了操作默认图的方法，在更高级的用例中，这些方法可能有用。例如：

+ tf.Graph 会定义 tf.Operation 对象的命名空间：单个图中的每个操作必须具有唯一名称。如果请求的名称已被占用，TensorFlow 将在操作名称上附加 "_1"、"_2" 等字符，以便确保名称的唯一性。通过使用多个明确创建的图，您可以更有效地控制为每个操作指定什么样的名称。

+ 默认图会存储与添加的每个 tf.Operation 和 tf.Tensor 有关的信息。如果程序创建了大量未连接的子图，更有效的做法是使用另一个 tf.Graph 构建每个子图，以便回收不相关的状态。

您可以安装另一个 tf.Graph 作为默认图（使用 tf.Graph.as_default 上下文管理器）：

In [10]:
g_1 = tf.Graph()
with g_1.as_default():
    # Operations created in this scope will be added to `g_1`.
    c = tf.constant("Node in g_1")
    
    # Sessions created in this scope will run operations from `g_1`.
    sess_1 = tf.Session()
    

g_2 = tf.Graph()
with g_2.as_default():
    # Operations created in this scope will be added to `g_2`.
    d = tf.constant("Node in g_2")
    
# Alternatively, you can pass a graph when constructing a <a href="../api_docs/python/tf/Session"><code>tf.Session</code></a>:
# `sess_2` will run operations from `g_2`.
sess_2 = tf.Session(graph=g_2)

assert c.graph is g_1
assert sess_1.graph is g_1

assert d.graph is g_2
assert sess_2.graph is g_2

In [11]:
# 要检查当前的默认图，请调用 tf.get_default_graph，它会返回一个 tf.Graph 对象：
g = tf.get_default_graph()
print(g.get_operations())

[<tf.Operation 'Const' type=Const>, <tf.Operation 'c' type=Const>, <tf.Operation 'c_1' type=Const>, <tf.Operation 'outer/c' type=Const>, <tf.Operation 'outer/inner/c' type=Const>, <tf.Operation 'outer/c_1' type=Const>, <tf.Operation 'outer/inner_1/c' type=Const>, <tf.Operation 'Const_1' type=Const>, <tf.Operation 'random_uniform/shape' type=Const>, <tf.Operation 'random_uniform/min' type=Const>, <tf.Operation 'random_uniform/max' type=Const>, <tf.Operation 'random_uniform/RandomUniform' type=RandomUniform>, <tf.Operation 'random_uniform/sub' type=Sub>, <tf.Operation 'random_uniform/mul' type=Mul>, <tf.Operation 'random_uniform' type=Add>, <tf.Operation 'Variable' type=VariableV2>, <tf.Operation 'Variable/Assign' type=Assign>, <tf.Operation 'Variable/read' type=Identity>, <tf.Operation 'MatMul' type=MatMul>, <tf.Operation 'Softmax' type=Softmax>, <tf.Operation 'Const_2' type=Const>, <tf.Operation 'random_uniform_1/shape' type=Const>, <tf.Operation 'random_uniform_1/min' type=Const>, <tf